# Données des objets perdus

In [1]:
import requests

In [2]:
def specific_url_lostitem(annee):
    URL = "https://ressources.data.sncf.com/"
    ressource = "api/records/1.0/search/?dataset=objets-trouves-restitution&q=&rows=10000&sort=date&refine.gc_obo_gare_origine_r_name=Lille+Europe&refine.date="
    month_format = f"{annee}"
    return URL + ressource + month_format

In [3]:
from model import LostItem
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

engine = create_engine('sqlite:///db.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

field_list = [
    ["type_objet", "gc_obo_type_c"],
    ["date_restitution", "gc_obo_date_heure_restitution_c"]
]

for annee in [2016,2017,2018,2019,2020,2021,2022]:
    my_request = requests.get(specific_url_lostitem(annee))
    print(f"request réalisée pour {annee}")
    for lost_item in my_request.json()["records"]:   
        lost_item_data= {}
        
        for field in field_list:    
            try: 
                lost_item_data[field[0]] = lost_item["fields"][field[1]]
            except KeyError:
                lost_item_data[field[0]]=None

        session.add(LostItem(date=lost_item["fields"]["date"],**lost_item_data))
    
    session.commit()

request réalisée pour 2016
request réalisée pour 2017
request réalisée pour 2018
request réalisée pour 2019
request réalisée pour 2020
request réalisée pour 2021
request réalisée pour 2022


In [4]:
import dateparser
print(dateparser.parse("last week"))

2022-12-28 11:12:44.248926


In [5]:
def request_lost_item(start_date,end_date):
    if isinstance(start_date,str): 
        import dateparser
        start_date = dateparser.parse(start_date)
    if isinstance(end_date,str): 
        import dateparser
        end_date = dateparser.parse(end_date)
    
    start_date = str(start_date.date())
    end_date =  str(end_date.date())
    URL = "https://ressources.data.sncf.com/"
    ressource = f"api/records/1.0/search/?dataset=objets-trouves-restitution&q=date:[{start_date}%20TO%20{end_date}]&rows=10000&sort=date&refine.gc_obo_gare_origine_r_name=Lille+Europe"
    
    my_request = requests.get(URL + ressource)

    for lost_item in my_request.json()["records"]:   
        lost_item_data= {}
        
        for field in field_list:    
            try: 
                lost_item_data[field[0]] = lost_item["fields"][field[1]]
            except KeyError:
                lost_item_data[field[0]]=None

        session.add(LostItem(date=lost_item["fields"]["date"],**lost_item_data))
    
    session.commit()

    return URL + ressource

In [6]:
from datetime import datetime
request_lost_item("last week", datetime.now())

'https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=date:[2022-12-28%20TO%202023-01-04]&rows=10000&sort=date&refine.gc_obo_gare_origine_r_name=Lille+Europe'